# Pet Tracker Data Analysis
Analysis of tag location data stored in Parquet format using DuckDB

In [ ]:
# Import libraries
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os

# Set style for better visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Initialize DuckDB connection
conn = duckdb.connect()
print(f"DuckDB version: {duckdb.__version__}")
print(f"Current directory: {os.getcwd()}")

## 1. Data Overview

In [ ]:
# Check directory structure
import glob

parquet_files = glob.glob('tag_data/**/*.parquet', recursive=True)
print(f"Total Parquet files: {len(parquet_files)}")
print(f"\nSample files:")
for f in parquet_files[:5]:
    print(f"  {f}")

In [ ]:
# Load all data into a DataFrame
query = """
SELECT 
    tag_id,
    name,
    datetime,
    timestamp_ms,
    latitude,
    longitude,
    accuracy,
    battery,
    is_inaccurate,
    location_hash,
    year,
    month,
    day
FROM 'tag_data/**/*.parquet'
ORDER BY tag_id, timestamp_ms DESC
"""

df = conn.execute(query).df()
print(f"Total records: {len(df)}")
print(f"Unique tags: {df['tag_id'].nunique()}")
print(f"Date range: {df['datetime'].min()} to {df['datetime'].max()}")
df.head()

## 2. Statistics Summary

In [ ]:
# Overall statistics
stats_query = """
SELECT 
    COUNT(DISTINCT tag_id) as total_tags,
    COUNT(*) as total_records,
    COUNT(DISTINCT location_hash) as unique_locations,
    ROUND(AVG(accuracy), 2) as avg_accuracy_m,
    ROUND(MIN(accuracy), 2) as min_accuracy_m,
    ROUND(MAX(accuracy), 2) as max_accuracy_m,
    COUNT(DISTINCT DATE(datetime)) as unique_days
FROM 'tag_data/**/*.parquet'
"""

stats = conn.execute(stats_query).df()
stats.T.rename(columns={0: 'Value'})

In [ ]:
# Per-tag statistics
tag_stats_query = """
SELECT 
    tag_id,
    name,
    COUNT(*) as records,
    MIN(datetime) as first_seen,
    MAX(datetime) as last_seen,
    ROUND(AVG(accuracy), 2) as avg_accuracy_m,
    SUM(CASE WHEN is_inaccurate THEN 1 ELSE 0 END) as inaccurate_count,
    MAX(battery) as last_battery
FROM 'tag_data/**/*.parquet'
GROUP BY tag_id, name
ORDER BY tag_id
"""

tag_stats = conn.execute(tag_stats_query).df()
tag_stats

## 3. Data Visualizations

In [ ]:
# Accuracy distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram of accuracy
axes[0].hist(df['accuracy'], bins=20, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Accuracy (meters)')
axes[0].set_ylabel('Count')
axes[0].set_title('Distribution of Location Accuracy')
axes[0].axvline(df['accuracy'].mean(), color='red', linestyle='--', label=f'Mean: {df["accuracy"].mean():.2f}m')
axes[0].legend()

# Box plot by tag
df.boxplot(column='accuracy', by='tag_id', ax=axes[1], grid=False)
axes[1].set_xlabel('Tag ID')
axes[1].set_ylabel('Accuracy (meters)')
axes[1].set_title('Accuracy Distribution by Tag')
plt.suptitle('')  # Remove default title

plt.tight_layout()
plt.show()

In [ ]:
# Time-based analysis
time_query = """
SELECT 
    strftime(datetime, '%H') as hour,
    COUNT(*) as count,
    COUNT(DISTINCT tag_id) as unique_tags
FROM 'tag_data/**/*.parquet'
GROUP BY hour
ORDER BY hour
"""

time_df = conn.execute(time_query).df()

if not time_df.empty:
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.bar(time_df['hour'], time_df['count'], color='skyblue', edgecolor='black')
    ax.set_xlabel('Hour of Day')
    ax.set_ylabel('Number of Location Records')
    ax.set_title('Location Updates by Hour of Day')
    ax.grid(axis='y', alpha=0.3)
    plt.show()
else:
    print("Not enough time-based data for hourly analysis")

In [ ]:
# Geographic visualization
import folium
from folium import plugins

# Create a map centered on the mean location
center_lat = df['latitude'].mean()
center_lon = df['longitude'].mean()

m = folium.Map(location=[center_lat, center_lon], zoom_start=15)

# Add markers for each tag's latest location
latest_locations = df.loc[df.groupby('tag_id')['timestamp_ms'].idxmax()]

for _, row in latest_locations.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=5,
        popup=f"{row['name']}<br>Accuracy: {row['accuracy']:.2f}m<br>{row['datetime']}",
        tooltip=row['name'],
        color='blue',
        fill=True,
        fillColor='lightblue'
    ).add_to(m)

# Add a heatmap of all locations
heat_data = [[row['latitude'], row['longitude']] for idx, row in df.iterrows()]
plugins.HeatMap(heat_data, radius=15).add_to(m)

print(f"Map centered at: {center_lat:.6f}, {center_lon:.6f}")
print(f"Showing {len(latest_locations)} tags on the map")
m

## 4. Data Quality Analysis

In [ ]:
# Data quality metrics
quality_query = """
SELECT 
    'Battery Status' as metric,
    CAST(battery AS VARCHAR) as value,
    COUNT(*) as count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) as percentage
FROM 'tag_data/**/*.parquet'
GROUP BY battery
UNION ALL
SELECT 
    'Accuracy Status' as metric,
    CASE WHEN is_inaccurate THEN 'Inaccurate' ELSE 'Accurate' END as value,
    COUNT(*) as count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) as percentage
FROM 'tag_data/**/*.parquet'
GROUP BY is_inaccurate
UNION ALL
SELECT 
    'Accuracy Range' as metric,
    CASE 
        WHEN accuracy < 10 THEN '< 10m'
        WHEN accuracy < 20 THEN '10-20m'
        WHEN accuracy < 30 THEN '20-30m'
        WHEN accuracy < 50 THEN '30-50m'
        ELSE '50m+'
    END as value,
    COUNT(*) as count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) as percentage
FROM 'tag_data/**/*.parquet'
GROUP BY 
    CASE 
        WHEN accuracy < 10 THEN '< 10m'
        WHEN accuracy < 20 THEN '10-20m'
        WHEN accuracy < 30 THEN '20-30m'
        WHEN accuracy < 50 THEN '30-50m'
        ELSE '50m+'
    END
ORDER BY metric, value
"""

quality_df = conn.execute(quality_query).df()
quality_df

In [ ]:
# Visualize quality metrics
metrics = quality_df['metric'].unique()
fig, axes = plt.subplots(1, len(metrics), figsize=(15, 4))

for i, metric in enumerate(metrics):
    metric_data = quality_df[quality_df['metric'] == metric]
    ax = axes[i] if len(metrics) > 1 else axes
    
    ax.pie(metric_data['count'], 
           labels=metric_data['value'], 
           autopct='%1.1f%%',
           startangle=90)
    ax.set_title(metric)

plt.tight_layout()
plt.show()

## 5. Partition Analysis

In [ ]:
# Analyze partition structure
partition_query = """
SELECT 
    year,
    month,
    day,
    COUNT(DISTINCT tag_id) as tags,
    COUNT(*) as records,
    ROUND(AVG(accuracy), 2) as avg_accuracy
FROM 'tag_data/**/*.parquet'
GROUP BY year, month, day
ORDER BY year, month, day
"""

partitions = conn.execute(partition_query).df()
print("Partition Summary:")
partitions

In [ ]:
# File size analysis
import os

file_sizes = []
for file in parquet_files:
    size = os.path.getsize(file)
    parts = file.split('/')
    tag_id = int(parts[1].split('=')[1]) if 'tag_id=' in parts[1] else None
    file_sizes.append({
        'file': file,
        'tag_id': tag_id,
        'size_kb': size / 1024
    })

size_df = pd.DataFrame(file_sizes)
print(f"Total storage: {size_df['size_kb'].sum():.2f} KB")
print(f"Average file size: {size_df['size_kb'].mean():.2f} KB")
print(f"\nSize by tag:")
size_df.groupby('tag_id')['size_kb'].sum().sort_values(ascending=False).head(10)

## 6. Custom Queries

In [ ]:
# Find tags with most movement (highest location variance)
movement_query = """
SELECT 
    tag_id,
    name,
    COUNT(*) as locations,
    ROUND(STDDEV(latitude) * 111000, 2) as lat_variance_m,
    ROUND(STDDEV(longitude) * 111000, 2) as lon_variance_m,
    ROUND(MAX(latitude) - MIN(latitude), 6) as lat_range,
    ROUND(MAX(longitude) - MIN(longitude), 6) as lon_range
FROM 'tag_data/**/*.parquet'
GROUP BY tag_id, name
HAVING COUNT(*) > 0
ORDER BY (lat_variance_m + lon_variance_m) DESC
LIMIT 10
"""

movement = conn.execute(movement_query).df()
print("Tags with most movement variance:")
movement

In [ ]:
# Query specific tag history
tag_to_query = 1  # Change this to query different tags

tag_history_query = f"""
SELECT 
    datetime,
    latitude,
    longitude,
    accuracy,
    battery,
    is_inaccurate
FROM 'tag_data/tag_id={tag_to_query}/**/*.parquet'
ORDER BY timestamp_ms DESC
LIMIT 10
"""

tag_history = conn.execute(tag_history_query).df()
print(f"Recent history for Tag {tag_to_query}:")
tag_history

In [ ]:
# Close DuckDB connection
conn.close()
print("Analysis complete!")